In [26]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [8]:
#ω
omega_l=[2,2,2.5,2.5,3,3,3.5,3.5,5,5,5,5,3.5,3.5,3,3,2,2,2,2] #angular velocity (rad/s) out put of left wheel encoder
omega_r=[2,2,2,2,2.5,2.5,3.5,3.5,5,5,5,5,4.5,4.5,3.5,3.5,2.5,2.5,2,2] #angular velocity (rad/s) out put of right wheel encoder

In [9]:
L=0.5 #meters
robot_width=2*L #meters
wheel_radius = 0.1 #meters
delta_time = 0.5 #seconds

In [10]:
init_pos=(0,0,0)

In [27]:
class differential_drive_robot():
    
    def __init__(self,robot_width,wheel_radius,init_pos,delta_time):
        '''
        the initialization of the mobile robot object
        '''
        
        self.v_l=[0]
        self.v_r=[0]
        self.del_s_l=[0]
        self.del_s_r=[0]
        self.delta_s=[0]
        self.delta_y=[init_pos[1]]
        self.delta_x=[init_pos[0]]
        self.theta=[init_pos[2]]
        self.time=[0]
        self.initial_theta=init_pos[2]
        self.current_theta=init_pos[2]
        
        self.current_pos=init_pos
        self.path=[init_pos]
        self.robot_width=robot_width
        self.wheel_radius=wheel_radius
        self.init_pos=init_pos
        self.delta_time=delta_time
    def robot_coordinates(self):
        delx=self.delta_s[len(self.delta_s)-1]*math.cos(self.theta[len(self.theta)-1]+self.current_theta/2)
        dely=self.delta_s[len(self.delta_s)-1]*math.sin(self.theta[len(self.theta)-1]+self.current_theta/2)
        self.delta_x.append(delx)
        self.delta_y.append(dely)
        
    def robot_orientation(self,deltaSLeft,deltaSRight):
        delta_theta=(deltaSRight-deltaSLeft)/self.robot_width
        
        theta_plus_delta_theta=self.current_theta+delta_theta
        
        self.current_theta=delta_theta
        
        self.theta.append(theta_plus_delta_theta)
        self.robot_coordinates()
            
    def displacement(self, vel_l,vel_r):
        delta_s_l=vel_l*self.delta_time
        delta_s_r=vel_r*self.delta_time
        
        self.time.append(self.time[len(self.time)-1]+self.delta_time)
        
        self.del_s_l.append(delta_s_l)
        self.del_s_r.append(delta_s_r)
        
        self.delta_s.append((delta_s_l+delta_s_r)/2)
        self.robot_orientation(delta_s_l,delta_s_r)
        
    def velocity(self,left_omega,right_omega):
        vel_l=left_omega*wheel_radius
        vel_r=right_omega*wheel_radius
        
        self.v_l.append(vel_l)
        self.v_r.append(vel_r)
        
        self.displacement(vel_l,vel_r)
            
        
        

In [28]:
if __name__ == '__main__':
    '''main'''
    mobile_robot=differential_drive_robot(robot_width, wheel_radius, init_pos,delta_time)
    for i in range(len(omega_l)):
        mobile_robot.velocity(omega_l[i],omega_r[i])
        
#     print(mobile_robot.v_l,mobile_robot.v_r)
    #print(mobile_robot.time, mobile_robot.delta_s)
    print(mobile_robot.theta)
    print(mobile_robot.delta_x)
    print(mobile_robot.delta_y)

[0, 0.0, 0.0, -0.024999999999999994, -0.04999999999999999, -0.05000000000000002, -0.050000000000000044, -0.025000000000000022, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04999999999999999, 0.09999999999999998, 0.07499999999999998, 0.04999999999999999, 0.04999999999999999, 0.04999999999999999, 0.024999999999999994, 0.0]
[0, 0.1, 0.1, 0.11242090770677986, 0.11228034495376116, 0.13723153272126365, 0.13723153272126365, 0.17494531534824798, 0.17500000000000002, 0.25, 0.25, 0.25, 0.25, 0.1994377636224415, 0.19843953344586582, 0.16187832648004244, 0.16218272048876614, 0.11228034495376116, 0.11228034495376116, 0.09996875162757027, 0.1]
[0, 0.0, 0.0, -0.0042177612999892795, -0.007026673257267771, -0.008588156203327282, -0.008588156203327285, -0.004374544285074662, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014985941454548466, 0.024934946677045534, 0.01420061322692792, 0.010149639149386783, 0.007026673257267771, 0.007026673257267771, 0.0024997395914712326, 0.0]
